# 场景分类案例

In [1]:
# !/usr/bin/env python3
# coding=utf-8
import os
import time

import dotmap
import yaml

from iobjectspy import open_datasource
from iobjectspy.ml.vision import Inference, DataPreparation
from iobjectspy.ml.vision import Trainer

Using TensorFlow backend.


## 设置输入数据路径

In [2]:
curr_dir = ''
data_dir = os.path.join(curr_dir, '..','..','example_data')
dom_path = os.path.join(data_dir, 'inference', 'landcover_infer.tif')

## 设置输出数据路径

In [3]:
out_dir = os.path.join(curr_dir, '..','..','out')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

## 制作SCENE数据,用于模型训练

In [4]:
input_data = os.path.join(data_dir, 'training','scene_cls_train_data','raw','sceneclassification.tif')
ds = open_datasource(
    os.path.join(data_dir, 'training','scene_cls_train_data','raw','sceneclassification.udbx'))
input_label = ds['DLTB']

label_class_field = 'DLMC'
output_path = os.path.join(curr_dir, '..','..','out')
output_name = 'SCENE'
start_time = time.time()
DataPreparation.create_training_data(input_data=input_data, input_label=input_label,
                                     label_class_field=label_class_field, output_path=output_path,
                                     output_name=output_name,
                                     training_data_format="SCENE_C", tile_format='origin', tile_size_x=128,
                                     tile_size_y=128,
                                     tile_offset_x=64,
                                     tile_offset_y=64, tile_start_index=0, save_nolabel_tiles=False)
print('完成，共耗时{}s，训练数据保存在 {}'.format(
    time.time() - start_time, os.path.join(output_path, output_name)))

java -cp /home/data/hou/workspaces/iobjectspy/venv/lib/python3.6/site-packages/iobjectspy-10.0.0-py3.6.egg/iobjectspy/_jsuperpy/jars/iobjects-py4j.jar com.supermap.jsuperpy.ApplicationExample 127.0.0.1 59861
[iObjectsPy]: Connection gateway-service successful, Python callback port bind 39201

完成，共耗时57.18492007255554s，训练数据保存在 ../../out/SCENE


## 场景分类模型训练

In [5]:
print('train start')
train_data_path = os.path.join(data_dir, 'training','scene_cls_train_data','train_data')
config = os.path.join(curr_dir, '..','..','trainer_config','scene_cls_train_config.sdt')
output_model_path = out_dir
output_model_name = 'example_scene_cls'
backbone_weight_path = os.path.join(curr_dir, '..','..','backbone','efficientnet-b1_imagenet_1000_notop.h5')

start_time = time.time()

Trainer(train_data_path=train_data_path, config=config, epoch=1, batch_size=1, lr=0.001,
            output_model_path=output_model_path,
            output_model_name='example_scene_cls',
            backbone_name='efficientnetb1',
            backbone_weight_path=backbone_weight_path).scene_classify_train()

end_time = time.time()
print('完成，共耗时{}s，结果保存在 {}'.format(
    end_time - start_time, os.path.join(output_model_path, output_model_name)))

train start
Train on 126 samples, validate on 41 samples
Epoch 1/1
126/126 [==============================] - 53s 418ms/step - loss: 9.0669 - acc: 0.3175 - categorical_crossentropy: 9.0669 - val_loss: 9.8281 - val_acc: 0.3902 - val_categorical_crossentropy: 9.8281

Epoch 00001: val_loss improved from inf to 9.82811, saving model to ./2019-09-19/scene_classification/checkpoints/scene_classification-0001-9.8281.hdf5
model saved in dir : ../../out/example_scene_cls/example_scene_cls.sdm
完成，共耗时121.00986170768738s，结果保存在 ../../out/example_scene_cls


## 设置模型路径

In [6]:
model_path = os.path.join(out_dir, 'example_scene_cls', 'example_scene_cls.sdm')

## 场景分类模型预测

In [7]:
out_data = os.path.join(out_dir, 'out_scene_cls.udb')
out_dataset_name = 'predict_scene'

start_time = time.time()
Inference(input_data=dom_path,
          model_path=model_path,
          out_data=out_data,
          out_dataset_name=out_dataset_name).scene_classify_infer(result_type='region')

end_time = time.time()
print('提取完成，共耗时{}s，结果数据保存在 {} 数据源下 {} 数据集中'.format(
    end_time - start_time, out_data,out_dataset_name))

/home/data/hou/workspaces/iobjectspy/venv/lib/python3.6/site-packages/rasterio/__init__.py:216: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100%,81The scene classification have done!
提取完成，共耗时9.693045616149902s，结果数据保存在 ../../out/out_scene_cls.udb 数据源下 predict_scene 数据集中
